# Cirrus fractions for DYAMOND2 models

1. "convective cirrus" = qtot(+500m) > ~~5e-5~~ 2.5e-4

2. "other stratospheric cirrus" = qtot(+500m) > 1e-6 and < ~~5e-5~~ 2.5e-4

^ try this....

#### want to know how often you have data point falling into one of the two categories

In [1]:
import pickle
import sys
sys.path.append("/home/b/b380887/cold-point-overshoot/python_scripts/")

import xarray as xr
import numpy as np
import get_d2_data as get_d2


In [2]:
file_path = "/work/bb1153/b380887/10x10/"
pickle_dir = "/home/b/b380887/cold-point-overshoot/pickle_files/d2_cirrus_fracs/new_thresh/"


In [3]:
conv_thresh = 2.5e-4 #5e-5

In [4]:
# model_list = ["GEOS"] #"SHIELD", "SCREAM"] #, GEOS] # OBS
model_list = ["SCREAM"]
region_list = ["AMZ", "SPC"]#, "TIM", "SCA"]


In [4]:
def get_conv_other_fracs(region, model, file_path, conv_thresh):
    """ Get the convective & other cirrus fractions
    """
    tb = get_d2.get_brightness_temp(region, model, file_path)
    cpT, cp_inds = get_d2.get_cold_point(region, model, file_path, return_inds=True)
    qtot = get_d2.get_qtot(region, model, file_path)

    if len(tb.time) < len(cpT.time):
        cpT = cpT.sel(time=tb.time)
    elif len(cpT.time) < len(tb.time):
        tb = tb.sel(time=cpT.time)
    diffs = tb - cpT

    if model == "SHIELD":
        # diffs = diffs.sel(time=qtot.time)
        cp_inds = cp_inds.sel(time=qtot.time)
        
    if model == "SHIELD":
        qtot500 = qtot.isel(pfull_ref=cp_inds+1)
    elif model == "SCREAM" or model == "GEOS":
        qtot500 = qtot.isel(lev=cp_inds+2)
        
    qtot500_conv = qtot500.where(qtot500 >= conv_thresh)
    qtot500_other = qtot500.where(qtot500 < conv_thresh).where(qtot500 >= 1e-6)
    n_cols = qtot500.size
    print(conv_thresh, n_cols)
    
    frac_conv = qtot500_conv.count().values / n_cols
    # frac_other = qtot500.count().values / n_cols
    frac_other = qtot500_other.count().values / n_cols

    return frac_conv, frac_other


In [5]:
def get_obs_conv_other_fracs(region, file_path, conv_thresh, a500_ref=0.136337):
    """ Get the convective & other cirrus fractions
    """
    qtot500 = get_d2.get_qtot(region, "OBS", file_path, obs_cp_relative=True)["iwc_a500"]/a500_ref

    qtot500_conv = qtot500.where(qtot500 >= conv_thresh)
    qtot500_other = qtot500.where(qtot500 < conv_thresh).where(qtot500 >= 1e-6)
    n_cols = qtot500.size
    
    frac_conv = qtot500_conv.count().values / n_cols
    # frac_other = qtot500.count().values / n_cols
    frac_other = qtot500_other.count().values / n_cols
    
    return frac_conv, frac_other


In [6]:
model = "GEOS"
region = "SPC"

In [7]:
frac_conv, frac_other = get_conv_other_fracs(region, model, file_path, conv_thresh=conv_thresh)

[########################################] | 100% Completed | 10.7s
0.00025 20520960


In [8]:
# new
print(frac_conv*100.)
print(frac_other*100.)

0.007519141404690618
0.0754594326971058


In [9]:
frac_conv2, frac_other2 = get_conv_other_fracs(region, model, file_path, conv_thresh=5e-5)

[########################################] | 100% Completed |  7.9s
5e-05 20520960


In [10]:
# old
print(frac_conv2*100.)
print(frac_other2*100.)

0.01952150386726547
0.06345707023453094


In [8]:
%%time
for region in region_list:
        frac_conv, frac_other = get_obs_conv_other_fracs(region, file_path, conv_thresh=conv_thresh)
        print("DARDAR", region)
        print("convective", frac_conv*100., "%")
        print("other", frac_other*100., "%\n")
        

*** returning DARDAR IWC, not mixing ratio ***
DARDAR AMZ
convective 0.031417466315987894 %
other 1.4622586465355512 %

*** returning DARDAR IWC, not mixing ratio ***
DARDAR SPC
convective 0.008134710811030668 %
other 7.069515623164041 %

CPU times: user 159 ms, sys: 37.7 ms, total: 197 ms
Wall time: 865 ms


In [7]:
%%time
for model in model_list:
    
    for region in region_list:
        frac_conv, frac_other = get_conv_other_fracs(region, model, file_path, conv_thresh=conv_thresh)
        print(model, region)
        print("convective", frac_conv*100., "%")
        print("other", frac_other*100., "%\n")
        
        frac_dict = {
            "convective": frac_conv,
            "other": frac_other,
            "conv_thresh": conv_thresh
        }
        
        with open(pickle_dir + "{m}_conv_other_fracs_{r}.pickle".format(m=model, r=region), "wb") as handle:
            pickle.dump(frac_dict, handle)
        

[########################################] | 100% Completed |  1min 10.9s
SHIELD AMZ
convective 0.02207252221222257 %
other 0.4451704114560583 %

[########################################] | 100% Completed |  3min 56.2s
SHIELD SPC
convective 0.05943789192899987 %
other 1.4161970762591924 %

SCREAM AMZ
convective 0.03023012180830285 %
other 0.2356484605277176 %

SCREAM SPC
convective 0.015540164610632541 %
other 0.38907168300622247 %

CPU times: user 28.8 s, sys: 46.4 s, total: 1min 15s
Wall time: 7min 20s


In [ ]:
# Just GEOS (memory issue before for AMZ!)

In [7]:
%%time
for model in model_list:
    
    for region in region_list:
        frac_conv, frac_other = get_conv_other_fracs(region, model, file_path, conv_thresh=conv_thresh)
        print(model, region)
        print("convective", frac_conv*100., "%")
        print("other", frac_other*100., "%\n")
        
        frac_dict = {
            "convective": frac_conv,
            "other": frac_other,
            "conv_thresh": conv_thresh
        }
        
        with open(pickle_dir + "{m}_conv_other_fracs_{r}.pickle".format(m=model, r=region), "wb") as handle:
            pickle.dump(frac_dict, handle)
        

[########################################] | 100% Completed | 45.5s
GEOS AMZ
convective 0.014604932592618804 %
other 0.33261974688321955 %

[########################################] | 100% Completed | 46.2s
GEOS SPC
convective 0.007519141404690618 %
other 0.0754594326971058 %

CPU times: user 34.3 s, sys: 53.9 s, total: 1min 28s
Wall time: 3min 25s
